A.Loading elasticsearch

In [2]:
from elasticsearch import Elasticsearch

# Adds the HTTP header 'Authorization: Basic <base64 username:password>'
es = Elasticsearch(
    "http://localhost:9200",
    basic_auth=("elastic", "lxDO2mQDVQPrxE3FRWAy")
)

print(es.ping())

True


In [44]:
es.indices.get_alias().keys()

<ipython-input-44-917cf5300b8f>:1: ElasticsearchWarning: this request accesses system indices: [.transform-internal-007, .kibana_8.4.3_001, .apm-agent-configuration, .kibana_security_session_1, .tasks, .security-7, .apm-custom-link, .kibana_task_manager_8.4.3_001, .security-profile-8, .async-search], but in a future major version, direct access to system indices will be prevented by default
  es.indices.get_alias().keys()


dict_keys(['.kibana-event-log-8.4.3-000001', 'ecommerce_data', 'a', 'comments', 'news_headlines', '.transform-internal-007', '.kibana_8.4.3_001', '.apm-agent-configuration', 'questions2', '.kibana_security_session_1', '.tasks', 'metrics-endpoint.metadata_current_default', '.security-7', 'favorite_candy', '.apm-custom-link', '.kibana_task_manager_8.4.3_001', '.security-profile-8', '.async-search', '.transform-notifications-000002', '.metrics-endpoint.metadata_united_default'])

In [45]:
es.indices.delete(index="questions")

ObjectApiResponse({'acknowledged': True})

B.Importing data for assignments(1,2,3)

In [3]:
import pandas as pd
df_q = pd.read_csv('C:\\Users\\A\\Music\\repos\\Information-Retrieval\\data\\questions.csv').fillna("")

In [47]:
df_q.head(2)

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,...,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense
0,115145,967227,2009-05-31 16:20:08,CommonsWare,2022-10-29 23:14:28,https://commonsware.com,"Pennsylvania, United States","<p>Mark Murphy, founder of CommonsWare, writes...",269231,13193,...,2022-08-19 16:46:16,2022-08-19 16:46:16,"How can I fix ""unexpected element <queries> fo...",<android><android-gradle-plugin><android-manif...,7,1,30,,,CC BY-SA 4.0
1,85371,358430,2009-03-31 21:38:26,sehe,2022-10-29 15:03:44,,Netherlands,<p>I'm a developer at heart.</p>\n<p>I love C+...,34058,16475,...,2021-03-06 02:51:07,2021-03-06 02:51:07,C++20 behaviour breaking existing code with eq...,<c++><c++17><c++20><spaceship-operator>,1,4,21,,,CC BY-SA 4.0


In [48]:
df_q.columns

Index(['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'ProfileImageUrl', 'EmailHash', 'AccountId', 'Id.1', 'PostTypeId',
       'AcceptedAnswerId', 'ParentId', 'CreationDate.1', 'DeletionDate',
       'Score', 'ViewCount', 'Body', 'OwnerUserId', 'OwnerDisplayName',
       'LastEditorUserId', 'LastEditorDisplayName', 'LastEditDate',
       'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate', 'ContentLicense'],
      dtype='object')

In [7]:
# df_q['Id']=users.id
len(df_q['Id'].unique())

51

In [8]:
# df_q['Id.1'] = posts.id
len(df_q['Id.1'].unique())

100

C.Indexing data based on the assignment

In [49]:
mapping = {
     "properties": {
            "ID": {"type": "integer"},
            "Body": {"type": "text"},
            "Tags": {"type": "text"}
            #"CreationDate": {"type": "date","format": "yyyy-MM-dd HH:mm:ss"},
            #"UpVotes": {"type": "integer"},
            #"Score": {"type": "integer"},
            #"OwnerUserId": {"type": "integer"},
         }}

es.indices.create(index="questions",mappings=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'questions'})

In [50]:
for i,row in df_q.iterrows():
  doc = {
      "ID" : row['Id.1'], # id.1=posts.id
      "Body" : row["Body"],
      "Tags" : (row["Tags"] if row["Tags"]!= 'non' else '')
      #"CreationDate" : row["CreationDate"], no need
      #"UpVotes" : row["UpVotes"], no need
      #"Score" : row["Score"],
      #"OwnerUserId" : row["OwnerUserId"],
  }
  
  es.index(index="questions",id=i,document=doc)

C_2.Checking indexed data

In [51]:
es.indices.get(index='questions')

ObjectApiResponse({'questions': {'aliases': {}, 'mappings': {'properties': {'Body': {'type': 'text'}, 'ID': {'type': 'integer'}, 'Tags': {'type': 'text'}}}, 'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'questions', 'creation_date': '1667249323318', 'number_of_replicas': '1', 'uuid': '4fAFSMx5SVOoXWRSs70NFg', 'version': {'created': '8040399'}}}}})

In [52]:
es.indices.refresh(index="questions")


ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [53]:
es.cat.count(index="questions", format="json")

ListApiResponse([{'epoch': '1667249376', 'timestamp': '20:49:36', 'count': '100'}])

In [ ]:
res = es.get(index="questions", id=0)
res

In [55]:
res['_source']['Tags']

'<android><android-gradle-plugin><android-manifest><manifest>'

In [56]:
res.keys()

dict_keys(['_index', '_id', '_version', '_seq_no', '_primary_term', 'found', '_source'])

In [58]:
resp = es.search(index="questions", query={"match":{"Body":"program"}})
print(resp.keys())
print(resp['hits'].keys())
print(resp['hits']['hits'][0].keys()) #resp['hits']['hits'] is a list of results
print(resp['hits']['hits'][0]['_source'].keys())

dict_keys(['took', 'timed_out', '_shards', 'hits'])
dict_keys(['total', 'max_score', 'hits'])
dict_keys(['_index', '_id', '_score', '_source'])
dict_keys(['ID', 'Body', 'Tags'])


In [59]:
# first result
resp['hits']['hits'][0]


{'_index': 'questions',
 '_id': '82',
 '_score': 3.8813283,
 '_source': {'ID': 61278180,
  'Body': '<p>Consider the following demonstrative program.</p>\n\n<pre><code>#include &lt;iostream&gt;\n\nint main()\n{\n    typedef float T;\n\n    0.f.T::~T();\n}\n</code></pre>\n\n<p>This program is compiled by <code>Microsoft Visual Studio Community 2019</code>.</p>\n\n<p>But <code>clang</code> and <code>gcc</code> issue an error like this</p>\n\n<pre><code>prog.cc:7:5: error: unable to find numeric literal operator \'operator""f.T\'\n    7 |     0.f.T::~T();\n      |     ^~~~~\n</code></pre>\n\n<p>If to write the expression like <code>( 0.f ).T::~T()</code> then all three compilers compile the program.</p>\n\n<p>So a question arises: is this record <code>0.f.T::~T()</code> syntactically valid? And if not, then what syntactical rule is broken? </p>\n',
  'Tags': '<c++><syntax><floating-point><c++17><pseudo-destructor>'}}

In [60]:
resp['hits']['hits'][0]['_source']

{'ID': 61278180,
 'Body': '<p>Consider the following demonstrative program.</p>\n\n<pre><code>#include &lt;iostream&gt;\n\nint main()\n{\n    typedef float T;\n\n    0.f.T::~T();\n}\n</code></pre>\n\n<p>This program is compiled by <code>Microsoft Visual Studio Community 2019</code>.</p>\n\n<p>But <code>clang</code> and <code>gcc</code> issue an error like this</p>\n\n<pre><code>prog.cc:7:5: error: unable to find numeric literal operator \'operator""f.T\'\n    7 |     0.f.T::~T();\n      |     ^~~~~\n</code></pre>\n\n<p>If to write the expression like <code>( 0.f ).T::~T()</code> then all three compilers compile the program.</p>\n\n<p>So a question arises: is this record <code>0.f.T::~T()</code> syntactically valid? And if not, then what syntactical rule is broken? </p>\n',
 'Tags': '<c++><syntax><floating-point><c++17><pseudo-destructor>'}

In [ ]:
resp['hits']['hits'][0]['_source']['Body']

In [ ]:
print(f"Got {resp['hits']['total']['value']} Hits:" )
for hit in resp['hits']['hits']: #resp['hits']['hits'] is a list of results 
    print(hit['_source']['ID'])

Assignment(1)

In [65]:
def search_text(text):
    results = es.search(index='questions',query={"match":{"Body":text}})
    all_hits = results['hits']['hits']

    # see how many "hits" it returned using the len() function
    print ("total hits :", results['hits']['total']['value'], "\n")
    
    # iterate the nested dictionaries inside the ["hits"]["hits"] list and print results
    if results['hits']['total']['value'] == 0:
        print('NOT FOUND')  
    else:
        for num, doc in enumerate(all_hits):
            print ("DOC ID:", doc["_id"], "\n", doc['_source']['Body'], "\n")

In [66]:
search_text('concatenation')

total hits : 1 

DOC ID: 76 
 <p>I have existing code such as this:</p>

<pre><code>String sql = "CREATE TABLE " + tableName + " (\n" +
        "  id_ UUID DEFAULT random_uuid() PRIMARY KEY ,\n" +
        "  when_ TIMESTAMP WITHOUT TIME ZONE NOT NULL\n" +
        "  duration_ STRING NOT NULL\n" +
        ");";
</code></pre>

<p>…in a project in IntelliJ 2020.1.1 (preview) Ultimate edition where the JDK settings are configured for language level <code>14 (Preview) - Records, patters, text blocks</code>. </p>

<p>So I expected IntelliJ to offer a conversion to text blocks among the items display when I click on the yellow light bulb that appears next to this line of code. But I see no such offer.</p>

<p><a href="https://i.stack.imgur.com/f18mc.png" rel="noreferrer"><img src="https://i.stack.imgur.com/f18mc.png" alt="screen shot of pop-up menu displayed by clicking on yellow light bulb next to this line of code"></a></p>

<p>I am surprised to not find such an offer, as the JetBrains comp

Assignment(2)

In [6]:
df_q['Id.1']

0     62969917
1     65648897
2     67289057
3     60114639
4     61590721
        ...   
95    63788646
96    64034156
97    68024935
98    65224327
99    59618213
Name: Id.1, Length: 100, dtype: int64

In [88]:
def search_id(number):
    results = es.search(index='questions',query={"match":{"ID":number}})
    all_hits = results['hits']['hits']
    print("number of results = ",results['hits']['total']['value'],"\n")
    if results['hits']['total']['value'] == 0:
        print('NOT FOUND')  
    else:
        for num, doc in enumerate(all_hits):
            print ("DOC ID:", doc["_id"], "\n", doc['_source']['Body'], "\n")

In [89]:
search_id(64034156)

number of results =  1 

DOC ID: 96 
 <p><em><strong>Short problem description:</strong></em> I had created two new AWS EC2 server instances on Redhat Linux and Ubuntu operating system. I was trying to login to it through SSH via Putty but was getting errors(detailed errors further in question). This is just a short description.</p>
<p><em><strong>Detailed problem description:</strong></em> I generated a new .pem file after creating a free tier Amazon EC2(Elastic Compute Cloud) server of Ubuntu 20 O.S version. Then I have used <code>PuttyGen</code> software in my Windows-10 O.S p.c. While trying to login through putty it was giving me an error as shown in the following error description, due to this I was not able to login to aws machines, I have further investigated the issue and all details of troubleshooting steps are mentioned in detailed manner in this self answered question(other suggestions/solutions are welcome).</p>
<p><em><strong>Error's description:</strong></em></p>
<p><cod

Assignment(3)

In [8]:
df_q['Tags'].unique()

array(['<android><android-gradle-plugin><android-manifest><manifest>',
       '<c++><c++17><c++20><spaceship-operator>', '<c#><stackalloc>',
       '<selenium><google-chrome><selenium-chromedriver><chromium><renderer>',
       '<c++><class><language-lawyer><c++20><standard-layout>',
       '<c++><aggregate-initialization>',
       '<c++><c++17><c++20><stdtuple><spaceship-operator>',
       '<android><alarmmanager>', '<javascript><typescript><firebase>',
       '<windows-terminal>', '<ios><swift><nsdiffabledatasourcesnapshot>',
       '<c++><c++14>', '<android><android-app-signing>',
       '<c#><oop><attributes><c#-9.0><record-classes>',
       '<java><java-stream><iterable>',
       '<macos><text-to-speech><macos-catalina><siri><nsspeechsynthesizer>',
       '<c#><nameof>', '<ios><swift><combine>', '<javafx>',
       '<arrays><swift>', '<android><android-workmanager><android-12>',
       '<android><android-permissions>',
       '<r><dplyr><data.table><tidyverse><recode>',
       '<c++

In [25]:
results = es.search(index='questions',query={"match":{"Tags":"android" }})


all_hits = results['hits']['hits']
print(results['hits']['total']['value'])
if results['hits']['total']['value'] == 0:
    print('NOT FOUND')  
else:
    for num, doc in enumerate(all_hits):
        print ("DOC ID:", doc["_id"], "--->", doc['_source']['Tags'], "\n")

13
DOC ID: 20 ---> <android><android-workmanager><android-12> 

DOC ID: 21 ---> <android><android-permissions> 

DOC ID: 71 ---> <android> 

DOC ID: 12 ---> <android><android-app-signing> 

DOC ID: 0 ---> <android><android-gradle-plugin><android-manifest><manifest> 

DOC ID: 7 ---> <android><alarmmanager> 

DOC ID: 29 ---> <android><storage-access-framework><android-11> 

DOC ID: 66 ---> <android><xml><parsing><apk><android-manifest> 

DOC ID: 65 ---> <android><google-play> 

DOC ID: 30 ---> <android><encrypted-shared-preference> 



In [11]:
results = es.search(index='questions',query={"bool": {"must": [{"term":{"Tags":"android" }}]}})


all_hits = results['hits']['hits']
print(results['hits']['total']['value'])
if results['hits']['total']['value'] == 0:
    print('NOT FOUND')  
else:
    for num, doc in enumerate(all_hits):
        print ("DOC ID:", doc["_id"], "--->", doc['_source']['Tags'], "\n")

13
DOC ID: 20 ---> <android><android-workmanager><android-12> 

DOC ID: 21 ---> <android><android-permissions> 

DOC ID: 71 ---> <android> 

DOC ID: 12 ---> <android><android-app-signing> 

DOC ID: 0 ---> <android><android-gradle-plugin><android-manifest><manifest> 

DOC ID: 7 ---> <android><alarmmanager> 

DOC ID: 29 ---> <android><storage-access-framework><android-11> 

DOC ID: 66 ---> <android><xml><parsing><apk><android-manifest> 

DOC ID: 65 ---> <android><google-play> 

DOC ID: 30 ---> <android><encrypted-shared-preference> 



In [99]:
results = es.search(index='questions',query={"bool": {"must_not": [{"match":{"Tags":"android c java ios windows javascript " }}]}})


all_hits = results['hits']['hits']
print(results['hits']['total']['value'])
if results['hits']['total']['value'] == 0:
    print('NOT FOUND')  
else:
    for num, doc in enumerate(all_hits):
        print ("DOC ID:", doc["_id"], "--->", doc['_source']['Tags'], "\n")

36
DOC ID: 3 ---> <selenium><google-chrome><selenium-chromedriver><chromium><renderer> 

DOC ID: 15 ---> <macos><text-to-speech><macos-catalina><siri><nsspeechsynthesizer> 

DOC ID: 18 ---> <javafx> 

DOC ID: 19 ---> <arrays><swift> 

DOC ID: 22 ---> <r><dplyr><data.table><tidyverse><recode> 

DOC ID: 31 ---> <performance><assembly><x86-16><bios><vga> 

DOC ID: 33 ---> <google-cloud-platform><google-cloud-storage><google-cloud-build><google-cloud-run> 

DOC ID: 36 ---> <swift><async-await><swift5> 

DOC ID: 39 ---> <swift><option-type><combine><swift-keypath> 

DOC ID: 42 ---> <python><selenium><google-chrome><selenium-chromedriver><webusb> 



In [22]:
a='<c++><c++17><c++20><spaceship-operator>'
a = a.replace('<',"")
a = a.replace('>'," ")
a.split(" ")[:-1]

['c++', 'c++17', 'c++20', 'spaceship-operator']

In [81]:
def tokenizer(tags):
    tags = tags.replace('<',"")
    tags = tags.replace('>'," ")
    return tags

In [77]:
tokenizer('<c++><c++17><c++20><spaceship-operator>')

'c++ c++17 c++20 spaceship-operator '

In [45]:
es.search(index='questions',query={"bool": {"must": {"match": {"Body":"android" }},"filter":{"ids":{ "values": [0] }}}})['hits']['total']['value']

1

In [47]:
es.get(index='questions',id=0)['_source']['ID']

62969917

In [119]:
# i is the doc id which starts from zero
i=0
try:
    while(True):
        tags = es.get(index='questions',id=i)['_source']['Tags']
        tokenized_tags = tokenizer(tags)
    
        res = es.search(index='questions',query={"bool": {"must_not": {"match": {"Body":tokenized_tags }},"filter":{"ids":{ "values": [i] }}}})
        value = res['hits']['total']['value'] # number of hits
        if value :
            ID = es.get(index='questions',id=i)['_source']['ID']
            print('doc id : ',i,'-----','question ID :',ID)
        i+=1
except:
    print('done')

doc id :  27 ----- question ID : 63075418
doc id :  32 ----- question ID : 60583049
doc id :  36 ----- question ID : 67954414
doc id :  40 ----- question ID : 64012057
doc id :  41 ----- question ID : 65614890
doc id :  49 ----- question ID : 60624851
doc id :  79 ----- question ID : 63646183
doc id :  82 ----- question ID : 61278180
doc id :  84 ----- question ID : 67619477
doc id :  88 ----- question ID : 66843397
done


In [117]:
IDs = []
for i in range(100):
    tags = es.get(index='questions',id=i)['_source']['Tags']
    tokenized_tags = tokenizer(tags)
    
    res = es.search(index='questions',query={"bool": {"must_not": {"match": {"Body":tokenized_tags }},"filter":{"ids":{ "values": [i] }}}})
    value = res['hits']['total']['value'] # number of hits
    if value :
        IDs.append(es.get(index='questions',id=i)['_source']['ID'])
        print(i)

27
32
36
40
41
49
79
82
84
88


In [70]:
es.get(index='questions',id=27)['_source']['Tags']

'<ios><uitableview><ios14>'

In [72]:
es.search(index='questions',query={"bool": {"must": {"match": {"Body":'ios' }},"filter":{"ids":{ "values": [27] }}}})

ObjectApiResponse({'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

In [73]:
es.search(index='questions',query={"bool": {"must": {"match": {"Body":'uitableview' }},"filter":{"ids":{ "values": [27] }}}})

ObjectApiResponse({'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

PART 2 : 

In [120]:
import pandas as pd
df_a = pd.read_csv('C:\\Users\\A\\Music\\repos\\Information-Retrieval\\data\\answers.csv').fillna("")

In [121]:
df_a.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'DeletionDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId',
       'OwnerDisplayName', 'LastEditorUserId', 'LastEditorDisplayName',
       'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount',
       'CommentCount', 'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate',
       'ContentLicense'],
      dtype='object')

In [127]:
len(df_a['ParentId'].unique())

97

In [128]:
es.indices.delete(index="answers")

ObjectApiResponse({'acknowledged': True})

In [129]:
mapping = {
     "properties": {
            "ParentId": {"type": "integer"},
            "CreationDate": {"type": "date","format": "yyyy-MM-dd HH:mm:ss"},
            "Score": {"type": "integer"},
            "Body": {"type": "text"},
         }}

es.indices.create(index="comments",mappings=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'comments'})

In [130]:
for i,row in df_a.iterrows():
  doc = {
      "ParentId" : row['ParentId'],
      "CreationDate" : row["CreationDate"],
      "Score" : row["Score"],
      "Body" : row["Body"],
  }
  es.index(index="answers",id=i,document=doc)

assignment 4

In [140]:
df_a['ParentId']

0      65648897
1      66497269
2      66497269
3      62496779
4      62496779
         ...   
240    60644544
241    63017136
242    63017136
243    69309346
244    60644544
Name: ParentId, Length: 245, dtype: int64

In [146]:
def searchByIdLowscore(id,score):
    results = es.search(index='answers',query={"bool": {"must": [{"term": {"ParentId": id}}],"filter":{"range":{"Score": {"gte": score}}}}})
    all_hits = results['hits']['hits']
    print(results['hits']['total']['value'],' results \n')
    if results['hits']['total']['value'] == 0:
        print('NOT FOUND')  
    else:
        for num, doc in enumerate(all_hits):
            print ("DOC ID:", doc["_id"], "--->","Score = ",doc['_source']['Score'],"\n" ,doc['_source']['Body'], "\n")

In [148]:
searchByIdLowscore(62496779,5)

5  results 

DOC ID: 3 ---> Score =  36 
 <p>Currently you cannot open an elevated <code>wt.exe</code> session from the command line without workarounds. Workarounds include using <a href="https://github.com/microsoft/terminal/issues/632#issuecomment-616740964" rel="noreferrer">gsudo</a>, Using <a href="https://github.com/microsoft/terminal/issues/632#issuecomment-616740964" rel="noreferrer">Task Scheduler</a> (I tested this one and it works but you need to use the full path to wt.exe and you can skip the shortcut creation step) OR if you are ok with a keyboard shortcut, the simplest way; using a keyboard shortcut to run Windows Terminal as Admin from the taskbar.</p>
<p>For your use case:</p>
<blockquote>
<p>For my specific instance, I simply want to make it simpler to pop open
an admin terminal, I don't need a way to elevate arbitrary commands,
then I will happily use the commands I have already shown here.</p>
</blockquote>
<p>The simplest approach will work:</p>
<p><strong>Pin Wind

In [136]:
results = es.search(index='answers',query={"bool": {"must": [{"term": {"ParentId": 65648897}}],"filter":{"range":{"Score": {"gte": 5}}}}})
all_hits = results['hits']['hits']
print(results['hits']['total']['value'],' results \n')
if results['hits']['total']['value'] == 0:
    print('NOT FOUND')  
else:
    for num, doc in enumerate(all_hits):
        print ("DOC ID:", doc["_id"], "--->","Score = ",doc['_source']['Score'],"\n" ,doc['_source']['Body'], "\n")

1  results 

DOC ID: 0 ---> Score =  88 
 <p>Indeed, C++20 unfortunately makes this code infinitely recursive.</p>
<p>Here's a reduced example:</p>
<pre class="lang-cpp prettyprint-override"><code>struct F {
    /*implicit*/ F(int t_) : t(t_) {}

    // member: #1
    bool operator==(F const&amp; o) const { return t == o.t; }

    // non-member: #2
    friend bool operator==(const int&amp; y, const F&amp; x) { return x == y; }

private:
    int t;
};
</code></pre>
<p>Let's just look at <code>42 == F{42}</code>.</p>
<p>In C++17, we only had one candidate: the non-member candidate (<code>#2</code>), so we select that. Its body, <code>x == y</code>, itself only has one candidate: the member candidate (<code>#1</code>) which involves implicitly converting <code>y</code> into an <code>F</code>. And then that member candidate compares the two integer members and this is totally fine.</p>
<p>In C++20, the initial expression <code>42 == F{42}</code> now has <em>two</em> candidates: both the no

assignment 5

In [ ]:
results = es.search(index='answers',query={"match_phrase":{"Body":"good behavior"}})
all_hits = results['hits']['hits']
print(results['hits']['total']['value'],' results \n')
if results['hits']['total']['value'] == 0:
    print('NOT FOUND')  
else:
    for num, doc in enumerate(all_hits):
        print ("DOC ID:", doc["_id"], "--->",doc['_source']['Body'], "\n")

assignment 6

In [157]:
results = es.search(index='answers',query={"bool": {"must": [{"match_phrase": {"Body":"python"}}],"filter":{"range":{"CreationDate": {"gte": "2020-09-27 10:37:31","lte": "2022-09-27 10:37:31"}}}}})
all_hits = results['hits']['hits']
print(results['hits']['total']['value'],' results \n')
if results['hits']['total']['value'] == 0:
    print('NOT FOUND')  
else:
    for num, doc in enumerate(all_hits):
        print ("DOC ID:", doc["_id"], "--->","CreationDate = ",doc['_source']['CreationDate'],"\n" ,doc['_source']['Body'], "\n")

3  results 

DOC ID: 189 ---> CreationDate =  2022-06-26 10:35:54 
 <p>Make sure u have installed a correct python version.</p>
<p>I have noted that it works on python version 3.7 or lower version.</p>
 

DOC ID: 184 ---> CreationDate =  2021-12-04 10:31:14 
 <p>This works for me in all python version.</p>
<pre><code>pip install webdriver-manager
</code></pre>
<p>Here's Lib, Check out:
<a href="https://pypi.org/project/webdriver-manager/" rel="nofollow noreferrer">https://pypi.org/project/webdriver-manager/</a></p>
 

DOC ID: 186 ---> CreationDate =  2021-12-16 03:03:04 
 <p>some times just restarting your code editor can be the ultimate solution
that is what worked for me after spending 2 hours on the web trying to find out why the webdriver_manager is not picked up by python</p>
<p>use this to install</p>
<pre><code>pip install webdriver_manager
</code></pre>
<p>and then restart your VScode if that is what you are using as a code editor
I don't know why I have to but it worked the ti

PART 3

In [158]:
import pandas as pd
df_c = pd.read_csv('C:\\Users\\A\\Music\\repos\\Information-Retrieval\\data\\comments.csv').fillna("")

In [159]:
df_c.columns

Index(['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserDisplayName',
       'UserId', 'ContentLicense'],
      dtype='object')

In [173]:
es.indices.delete(index="comments")

ObjectApiResponse({'acknowledged': True})

In [174]:
mapping = {
     "properties": {
            "Id": {"type": "integer"},
            "PostId": {"type": "integer"},
            #"CreationDate": {"type": "date","format": "yyyy-MM-dd HH:mm:ss"}, #no need
            #"Score": {"type": "integer"},
            "Body": {"type": "text"},
         }}

es.indices.create(index="comments",mappings=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'comments'})

In [175]:
for i,row in df_c.iterrows():
    doc = {
        "Id" : row['Id'],
        "PostId" : row['PostId'],
        "Body" : row["Text"]
    }
    es.index(index="comments",id=i,document=doc)
    

In [176]:
es.get(index='comments',id=0)

ObjectApiResponse({'_index': 'comments', '_id': '0', '_version': 1, '_seq_no': 0, '_primary_term': 1, 'found': True, '_source': {'Id': 105375001, 'PostId': 59603086, 'Body': 'The question is answered in [Is is valid to construct std::set of pointer type?](https://stackoverflow.com/questions/53927571/is-is-valid-to-construct-stdset-of-pointer-type)'}})

assignment 7


In [231]:
for i in range(100):
    ID = es.get(index='questions',id=i)['_source']['ID']     # id of the question
    tags = es.get(index='questions',id=i)['_source']['Tags'] # tags of the question
    tokenized_tags = tokenizer(tags)

    # finding comments of the question
    res_comments = es.search(index='comments',query={"bool": {"must": {"match": {"PostId":ID}}}})
    number_comments = res_comments['hits']['total']['value'] # number of comments for the question
    for j in range(number_comments):
        # we have to define "size" because size_limit is 10 by default and does not show more than that
        comment_doc_id = es.search(index='comments',size=1000,query={"bool": {"must": {"match": {"PostId":ID}}}})['hits']['hits'][j]['_id']
        res_tag = es.search(index='comments',query={"bool": {"must": {"match": {"Body":tokenized_tags }},"filter":{"ids":{ "values": [comment_doc_id] }}}})
        value_tag = res_tag['hits']['total']['value'] # number of hits
        if value_tag :
            print('doc id : ',i,'-----','question ID :',ID)
            break
            

doc id :  0 ----- question ID : 62969917
doc id :  1 ----- question ID : 65648897
doc id :  3 ----- question ID : 60114639
doc id :  5 ----- question ID : 60150749
doc id :  6 ----- question ID : 66497269
doc id :  7 ----- question ID : 60079472
doc id :  10 ----- question ID : 64081701
doc id :  11 ----- question ID : 64977541
doc id :  12 ----- question ID : 63705739
doc id :  13 ----- question ID : 63778276
doc id :  14 ----- question ID : 59869486
doc id :  15 ----- question ID : 61122378
doc id :  16 ----- question ID : 69116668
doc id :  18 ----- question ID : 61531317
doc id :  19 ----- question ID : 68332664
doc id :  20 ----- question ID : 67749284
doc id :  21 ----- question ID : 69530502
doc id :  22 ----- question ID : 67081496
doc id :  23 ----- question ID : 60442205
doc id :  26 ----- question ID : 62550828
doc id :  28 ----- question ID : 63421086
doc id :  29 ----- question ID : 65967690
doc id :  31 ----- question ID : 61521819
doc id :  32 ----- question ID : 6058304

TEST

In [247]:
#id=24 not in results
es.get(index='questions',id=24)['_source']['ID']

61536203

In [249]:
es.get(index='questions',id=24)['_source']['Tags']

'<javascript><node.js><firebase-realtime-database>'

In [248]:
es.search(index='comments',query={"bool": {"must": {"match": {"PostId":61536203}}}})

ObjectApiResponse({'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'comments', '_id': '41', '_score': 1.0, '_source': {'Id': 109971099, 'PostId': 61536203, 'Body': 'Does the same rules apply to firestore too? Having increment call in a write batch assures the data is written without any problems? (Assuming 1write/sec/per doc limit)'}}]}})

In [250]:
es.get(index='questions',id=0)['_source']['ID']

62969917

In [251]:
es.get(index='questions',id=0)['_source']['Tags']

'<android><android-gradle-plugin><android-manifest><manifest>'

In [252]:
es.search(index='comments',query={"bool": {"must": {"match": {"PostId":62969917}}}})

ObjectApiResponse({'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'comments', '_id': '548', '_score': 1.0, '_source': {'Id': 130428881, 'PostId': 62969917, 'Body': 'yes it is working in case <query. tag in manifest classpath("com.android.tools.build:gradle:3.5.3")\n\nCurrent setting:\n\nclasspath("com.android.tools.build:gradle:3.5.4")'}}]}})